In [ ]:
# Library imports
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pennylane as qml
from sklearn import datasets
import tensorflow as tf
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from scipy.linalg import sqrtm

# Pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

import time
import datetime
from alive_progress import alive_bar

# Set the random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_path = 'torch_results/QGAN/QGAN_linear/' + current_time + '/'
summary_writer = tf.summary.create_file_writer(save_path)

### Classical data

In [ ]:
# Load the digits dataset
digits = datasets.load_digits()

x_train = digits.data
y_train = digits.target

x_train = x_train.reshape(len(x_train), 8, 8)
x_train.shape

In [ ]:
def resize_data(x, y, label, image_size):

    arr = []
    arr_input = []

    for t, l in zip(x, y):
        if l in label:
            t = torch.tensor(t, dtype = torch.float32).reshape(image_size, image_size)
            t = t/16
            arr.append((t, l))
            arr_input.append(t)
    return arr, arr_input

rd, inp = resize_data(x_train, y_train, label = (0, 1), image_size = 8)
#rd

In [ ]:
image_size = 8  # Height / width of the square images
batch_size = 1

dataloader = torch.utils.data.DataLoader(rd, batch_size=batch_size, shuffle=True, drop_last=True)

#for element in dataloader1:
#   print(element)

In [ ]:
plt.figure(figsize=(8,2))

for i in range(8):
    image = rd[i][0].reshape(image_size,image_size)
    plt.subplot(1,8,i+1)
    plt.axis('off')
    plt.imshow(image.numpy(), cmap='gray')
    
plt.show() 

In [ ]:
torch.save(inp, save_path + 'real.pt')

### Quantum Generator

In [ ]:
# Quantum variables
n_qubits = 5  
ancillary_qubits = 1 
gen_n_layers = 10  
n_generators = 4  
image_size = 8

In [ ]:
# Quantum simulator
dev = qml.device("default.qubit", wires=n_qubits)
device = torch.device("cpu")
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
@qml.qnode(dev, interface="torch", diff_method="parameter-shift")
def quantum_generator_circuit(noise, gen_weights):

    gen_weights = gen_weights.reshape(gen_n_layers, n_qubits)

    # Encoding layer
    for i in range(n_qubits):
        qml.RY(noise[i], wires=i)

    # PQC layers
    for i in range(gen_n_layers):

        # Rotation gates
        for y in range(n_qubits):
            qml.RY(gen_weights[i][y], wires=y)

        # Entangling gates
        for y in range(n_qubits - 1):
            qml.CZ(wires=[y, y + 1])

    # Returning probability of each computational basis state
    return qml.probs(wires=list(range(n_qubits)))

In [ ]:
# Testing generator circuit

noise = torch.rand(n_qubits, device=device) * math.pi / 2
gen_weights = torch.rand(gen_n_layers, n_qubits, device=device) * math.pi / 2

qgc = quantum_generator_circuit(noise, gen_weights)
qgc 

In [ ]:
# Showing generator circuit
fig, ax = qml.draw_mpl(quantum_generator_circuit)(noise, gen_weights)
plt.show()

In [ ]:
def partial_measure(noise, weights):
    probs = quantum_generator_circuit(noise, weights)
    partial_meas = probs[: (2 ** (n_qubits - ancillary_qubits))]
    partial_meas /= torch.sum(probs)

    # Post-Processing
    probsgiven = partial_meas / torch.max(partial_meas)
    return probsgiven

In [ ]:
class QuantumGenerator(nn.Module):

    def __init__(self, n_qubits, ancillary_qubits, gen_n_layers, n_generators, q_delta=1):

        self.n_qubits = n_qubits
        self.ancillary_qubits = ancillary_qubits
        self.gen_n_layers = gen_n_layers
        self.n_generators = n_generators

        super().__init__()

        self.q_params = nn.ParameterList([nn.Parameter(q_delta * torch.rand(self.gen_n_layers * self.n_qubits), 
                                          requires_grad=True)for _ in range(self.n_generators)])

    def forward(self, x):
        
        patch_size = 2 ** (n_qubits - ancillary_qubits)

        images = torch.Tensor(x.size(0), 0).to(device)

        # Iterate over all sub-generators
        for params in self.q_params:

            
            patches = torch.Tensor(0, patch_size).to(device)
            for elem in x:

                probs = quantum_generator_circuit(elem, params)
                partial_meas = probs[: (2 ** (n_qubits - ancillary_qubits))]
                partial_meas /= torch.sum(probs)

            
                out = partial_meas / torch.max(partial_meas)
                out = out.float().unsqueeze(0)
                patches = torch.cat((patches, out))

            # define the image
            images = torch.cat((images, patches), 1)

        return images

In [ ]:
# Testing quantum generator
fixed_noise = torch.rand(1, n_qubits, device=device) #* np.pi / 2

generator = QuantumGenerator(n_qubits = 5, ancillary_qubits = ancillary_qubits, 
                                  gen_n_layers = gen_n_layers, n_generators = 4).to(device)
test_images = generator(fixed_noise).view(1,1,8,8).cpu().detach()
test_images.shape

In [ ]:
images = torch.squeeze(test_images, dim=1)
for j, im in enumerate(images):
    plt.imshow(im.numpy(), cmap="gray")
    
plt.show()

In [ ]:
generator.parameters

### Discriminator Network

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, image_size):
        super().__init__()

        self.image_size = image_size

        self.model = nn.Sequential(
                                    # Inputs to first hidden layer (num_input_features -> 64)
                                    nn.Linear(self.image_size * self.image_size, 64),
                                    nn.ReLU(),
                                    # First hidden layer (64 -> 16)
                                    nn.Linear(64, 16),
                                    nn.ReLU(),
                                    # Second hidden layer (16 -> output)
                                    nn.Linear(16, 1),
                                    nn.Sigmoid(),
                                    )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Testing discriminator network

real_data = rd[0][0]
fake_data = test_images

disc = Discriminator(image_size)

In [ ]:
summary(disc, input_size=(64,))

### Training class

In [ ]:
class QGAN():
    def __init__(self, dataloader, gen_net, disc_net, n_qubits, image_size, batch_size, lrG, lrD, gen_loss, disc_loss):

        self.dataloader = dataloader
        self.gen_net = gen_net
        self.disc_net = disc_net
        self.n_qubits = n_qubits
        self.image_size = image_size
        self.batch_size = batch_size
        self.lrG = lrG
        self.lrD = lrD
        self.gen_loss = gen_loss
        self.disc_loss = disc_loss

        # Optimisers
        self.optD = optim.SGD(self.disc_net.parameters(), lr=self.lrD)
        self.optG = optim.SGD(self.gen_net.parameters(), lr=self.lrG)

        self.real_labels = torch.full((self.batch_size,), 1.0, dtype=torch.float, device=device)
        self.fake_labels = torch.full((self.batch_size,), 0.0, dtype=torch.float, device=device)  

        self.loss_g, self.loss_d = [], []  
        self.total_fid = []                       

    def generated_and_save_images(self, results):

        fig = plt.figure(figsize=(20, 10))
        outer = gridspec.GridSpec(5, 2, wspace=0.1)

        for i, images in enumerate(results):
            inner = gridspec.GridSpecFromSubplotSpec(1, images.size(0), subplot_spec=outer[i])
            
            images = torch.squeeze(images, dim=1)
            for j, im in enumerate(images):

                ax = plt.Subplot(fig, inner[j])
                ax.imshow(im.numpy(), cmap="gray")
                ax.set_xticks([])
                ax.set_yticks([])
                if j==0:
                    ax.set_title(f'Iteration {50+i*50}', loc='left', color = 'White')
                fig.add_subplot(ax)

        plt.show()

    def calculate_fid(self, dist1, dist2):

        # calculate mean and covariance statistics
        mu1, sigma1 = dist1.mean(axis=0), cov(dist1, rowvar=False)
        mu2, sigma2 = dist2.mean(axis=0), cov(dist2, rowvar=False)
        
        # calculate sum squared difference between means
        ssdiff = torch.sum((mu1 - mu2)**2.0)
        # calculate sqrt of product between cov
        covmean = sqrtm(sigma1.dot(sigma2))

        # check and correct imaginary numbers from sqrt
        if iscomplexobj(covmean):
            covmean = covmean.real
            
        # calculate score
        fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
        return fid

    def train_step(self, data):

        data = data.reshape(-1, self.image_size * self.image_size)
        real_data = data.to(device)

        noise = torch.rand(self.batch_size, self.n_qubits, device=device) 
        fake_data = self.gen_net(noise)

        # Discriminator training
        self.disc_net.zero_grad()
        outD_real = self.disc_net(real_data).view(-1)
        outD_fake = self.disc_net(fake_data.detach()).view(-1)

        errD_real = self.disc_loss(outD_real, self.real_labels)
        errD_fake = self.disc_loss(outD_fake, self.fake_labels)
     
        errD_real.backward()
        errD_fake.backward()

        errD = errD_real + errD_fake
        self.optD.step()

        # Generator training
        self.gen_net.zero_grad()
        outD_fake = self.disc_net(fake_data).view(-1)
        errG = self.gen_loss(outD_fake, self.real_labels)

        errG.backward()
        self.optG.step()

        return errG, errD


    def learn(self, epochs):

        
        self.fixed_noise = torch.rand(8, self.n_qubits, device=device) 

        
        epoch = 0      

        results = []

        with alive_bar(epochs, force_tty = True) as bar:

            while True:
                
                for _, (data, _) in enumerate(self.dataloader):

                    lg, ld = self.train_step(data)                
                    
                    epoch += 1

                    #time.sleep(0.05) 
                    bar()                   

                    # Show loss values         
                    if epoch % 10 == 0:
                        #print(f'Iteration: {epoch}, Generator Loss: {lg:0.3f}, Discriminator Loss: {ld:0.3f}')
                        test_images = self.gen_net(self.fixed_noise).view(8,1,self.image_size,self.image_size).cpu().detach()
                        
                        # Save images every 50 iterations
                        if epoch % 50 == 0:
                            results.append(test_images)  
                            torch.save(results, save_path + 'synthetic.pt')     
                            fid = self.calculate_fid(data[0], results[0][0][0])  
                            self.total_fid.append(fid.item())                               
                    
                    self.loss_g.append(lg.detach().numpy())
                    self.loss_d.append(ld.detach().numpy())      

                    if epoch == epochs:
                        break
                if epoch == epochs:
                    break                  
                           
            #torch.save(loss_g, save_path + 'gen_loss.pt') 
            #torch.save(loss_d, save_path + 'disc_loss.pt')
            #torch.save(self.total_fid, save_path + 'fid.pt')  
        
        # self.generated_and_save_images(results)
        # plt.figure(figsize=(11, 7))
        # plt.plot(np.arange(epochs), self.loss_g, color = 'Red', label = 'Generator Loss\n120 parameters')
        # plt.plot(np.arange(epochs), self.loss_d, color = 'Blue', label = 'Discriminator Loss\n5217 parameters')
        # plt.xlabel('Epochs', fontsize = 16)
        # plt.ylabel('Loss', fontsize = 16)
        # plt.legend(loc = 'upper right', fontsize = 16)
        # plt.show()

In [ ]:
n_qubits = 5
ancillary_qubits = 1
gen_n_layers = 10
n_generators = 4
image_size = 8
batch_size = 1
loss = nn.BCELoss()
lrG = 0.3
lrD = 0.01
epochs = 500

In [ ]:
gen_net = QuantumGenerator(n_qubits = n_qubits, ancillary_qubits = ancillary_qubits, 
                           gen_n_layers = gen_n_layers, n_generators = n_generators).to(device)
disc_net = Discriminator(image_size).to(device)

runs = 10

loss_g_mean = []
loss_d_mean = []
fid_mean = []

for run in range(runs): 

    qgan = QGAN(dataloader = dataloader, gen_net = gen_net, disc_net = disc_net, n_qubits = n_qubits, image_size = image_size, 
            batch_size = batch_size, lrG = lrG, lrD = lrD, gen_loss = loss, disc_loss = loss)

    qgan.learn(epochs)

    loss_g_mean.append(qgan.loss_g)
    loss_d_mean.append(qgan.loss_d)
    fid_mean.append(qgan.total_fid)
    
    torch.save(loss_g_mean, save_path + 'gen_loss.pt') 
    torch.save(loss_d_mean, save_path + 'disc_loss.pt') 
    torch.save(fid_mean, save_path + 'fid.pt')